## Idea

I want to use two out of the box features: chroma_stft and mfcc from librosa. For each feature, I will make a simple logistic regression model, then combine the output from both models as new features to train another model on top of it. 

Several friends had commented in the past that stacked/ensemble models tend to work better than a single model, so I want to try the workflow here.

![stacked](../../docs/images/chang-1.4-stacked-model.png)

Also got the idea off Ch.50 page 3 of Andrew Ng's book Deep Learning Yearning he's sending out in emails

The results are evaluated by a 75%-25% train-test split with the out of the box metric (mean accuracy).

In [1]:
import helpers
import pandas as pd
import pickle
import numpy as np
import math
import librosa.feature as lf
from sklearn import model_selection
from sklearn import linear_model

## Loading data for later use

**Can skip reading this whole section** - basically load and process it using Carlos's function to either read the whole audio or only the first second.

Somehow I always have memory error in reading Applauses when I try to parse the whole thing. It may have to do with the 32-bit Python installation I have on Windows 10.

In [2]:
TRAIN_CSV = 'data/external/train.csv'
TRAIN_FILES = 'data/external/audio_train'

In [3]:
tags = list(pd.read_csv(TRAIN_CSV).label.unique())

Load each label and process into 1 second clips separately

In [7]:
for tag_num, tag_name in enumerate(tags):
    train_files = helpers.find_paths_with_tags(csv_path=TRAIN_CSV, files_path=TRAIN_FILES, filters=[tag_name])
    wav_data = helpers.load_wav_files(train_files, duration=15)
    with open('data/raw/train/wav-data-{}.pkl'.format(tag_name), 'wb') as f:
        pickle.dump(wav_data, f)
    print('Processed {}-{}'.format(tag_num, tag_name))

Processed 0-Hi-hat
Processed 1-Saxophone
Processed 2-Trumpet
Processed 3-Glockenspiel
Processed 4-Cello
Processed 5-Knock
Processed 6-Gunshot_or_gunfire
Processed 7-Clarinet
Processed 8-Computer_keyboard
Processed 9-Keys_jangling
Processed 10-Snare_drum
Processed 11-Writing
Processed 12-Laughter
Processed 13-Tearing
Processed 14-Fart
Processed 15-Oboe
Processed 16-Flute
Processed 17-Cough
Processed 18-Telephone
Processed 19-Bark
Processed 20-Chime
Processed 21-Bass_drum
Processed 22-Bus
Processed 23-Squeak
Processed 24-Scissors
Processed 25-Harmonica
Processed 26-Gong
Processed 27-Microwave_oven
Processed 28-Burping_or_eructation
Processed 29-Double_bass
Processed 30-Shatter
Processed 31-Fireworks
Processed 32-Tambourine
Processed 33-Cowbell
Processed 34-Electric_piano
Processed 35-Meow
Processed 36-Drawer_open_or_close


MemoryError: 

In [4]:
for tag_num, tag_name in enumerate(tags[37:]):
    train_files = helpers.find_paths_with_tags(csv_path=TRAIN_CSV, files_path=TRAIN_FILES, filters=[tag_name])
    wav_data = helpers.load_wav_files(train_files, duration=15)
    with open('data/raw/train/wav-data-{}.pkl'.format(tag_name), 'wb') as f:
        pickle.dump(wav_data, f)
    print('Processed {}-{}'.format(tag_num, tag_name))

Processed 0-Applause
Processed 1-Acoustic_guitar
Processed 2-Violin_or_fiddle
Processed 3-Finger_snapping


#### 1 second versions

In [18]:
for tag_num, tag_name in enumerate(tags):
    train_files = helpers.find_paths_with_tags(csv_path=TRAIN_CSV, files_path=TRAIN_FILES, filters=[tag_name])
    wav_data = helpers.load_wav_files(train_files, duration=1)
    with open('data/raw/train-1-sec/wav-data-{}.pkl'.format(tag_name), 'wb') as f:
        pickle.dump(wav_data, f)
    print('Processed {}-{}'.format(tag_num, tag_name))

Processed 0-Hi-hat
Processed 1-Saxophone
Processed 2-Trumpet
Processed 3-Glockenspiel
Processed 4-Cello
Processed 5-Knock
Processed 6-Gunshot_or_gunfire
Processed 7-Clarinet
Processed 8-Computer_keyboard
Processed 9-Keys_jangling
Processed 10-Snare_drum
Processed 11-Writing
Processed 12-Laughter
Processed 13-Tearing
Processed 14-Fart
Processed 15-Oboe
Processed 16-Flute
Processed 17-Cough
Processed 18-Telephone
Processed 19-Bark
Processed 20-Chime
Processed 21-Bass_drum
Processed 22-Bus
Processed 23-Squeak
Processed 24-Scissors
Processed 25-Harmonica
Processed 26-Gong
Processed 27-Microwave_oven
Processed 28-Burping_or_eructation
Processed 29-Double_bass
Processed 30-Shatter
Processed 31-Fireworks
Processed 32-Tambourine
Processed 33-Cowbell
Processed 34-Electric_piano
Processed 35-Meow
Processed 36-Drawer_open_or_close
Processed 37-Applause
Processed 38-Acoustic_guitar
Processed 39-Violin_or_fiddle
Processed 40-Finger_snapping


## Process features

Here I processed the files to extract their features.

If an audio is less than 1 second long, then pad it to 1 second before processing it.

In [76]:
def pad_audio(sound: np.ndarray, sample_rate=22050):
    padded_sound = np.tile(sound, math.ceil(sample_rate / sound.shape[0]))
    return padded_sound[:sample_rate]

#### chroma_stft

For each label, read in the 1 second version, process with librosa.features.chroma_stft, then save it back to disk.

In [69]:
%%time
feature_name = 'chroma_stft'
for tag_num, tag_name in enumerate(tags):
    with open('data/raw/train-1-sec/wav-data-{}.pkl'.format(tag_name), 'rb') as f:
        wav_data = pickle.load(f)
    wav_features = {sample.name: lf.chroma_stft(pad_audio(sample.wav[0])).flatten() for sample in wav_data}
    df_features = (
        pd.DataFrame.from_dict(wav_features, orient='index')
        .reset_index().rename({'index': 'name'}, axis=1)
    )
    df_features.columns = ['name'] + [feature_name + '_' + str(column_name) for column_name in list(df_features.columns)[1:]]
    df_features.to_pickle('data/interim/{}-1-sec/{}.pkl'.format(feature_name, tag_name))

c:\users\chang\.virtualenvs\kaggle-audio-tagging-pu_5w54x\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Wall time: 1min 6s


#### mfcc

For each label, read in the 1 second version, process with librosa.features.mfcc, then save it back to disk.

In [45]:
%%time
feature_name = 'mfcc'
for tag_num, tag_name in enumerate(tags):
    with open('data/raw/train-1-sec/wav-data-{}.pkl'.format(tag_name), 'rb') as f:
        wav_data = pickle.load(f)
    wav_features = {sample.name: lf.mfcc(pad_audio(sample.wav[0])).flatten() for sample in wav_data}
    df_features = (
        pd.DataFrame.from_dict(wav_features, orient='index')
        .reset_index().rename({'index': 'name'}, axis=1)
    )
    df_features.columns = ['name'] + [feature_name + '_' + str(column_name) for column_name in list(df_features.columns)[1:]]
    df_features.to_pickle('data/interim/{}-1-sec/{}.pkl'.format(feature_name, tag_name))

Wall time: 1min 14s


## Make two models

One for each feature

#### Label the files 

Because scikit learn takes in numeric class labels only, I need to convert the string labels to numbers. 

EDIT: Only after I did it I found out about [sklearn.preprocessing.LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), so I will use that instead in the future.

In [4]:
label_reference = pd.Series(tags).rename('label').to_frame().reset_index().rename({'index': 'label_index'}, axis=1)

In [5]:
file_labels = (
    pd.read_csv(TRAIN_CSV)
    .drop(['manually_verified'], axis=1)
    .rename({'fname': 'name'}, axis=1)
    .merge(label_reference, on=['label'])
)

#### Split the data into training and test set

For both models, I need to split it into a training set and a test set so I can score later.

### chroma_stft

##### train-test split

In [21]:
# load all features from processed files
train_chroma_stft = pd.concat([pd.read_pickle('data/interim/{}-1-sec/{}.pkl'.format('chroma_stft', name)) for name in tags])

df_train_chroma_stft = (
    train_chroma_stft
    .merge(file_labels, on=['name'], how='inner')
)

X_chroma_train, X_chroma_test, y_chroma_train, y_chroma_test = model_selection.train_test_split(
    df_train_chroma_stft.drop(['name', 'label', 'label_index'], axis=1).values,
    df_train_chroma_stft['label_index'].values,
    test_size=0.25, 
    random_state=707,
    stratify=df_train_chroma_stft['label_index'].values
)

Use the multiclass version of logistic regression, don't balance class weight yet

##### train model

In [174]:
chroma_model = linear_model.LogisticRegression(random_state=123)

In [176]:
%%time
chroma_model.fit(X_chroma_train, y_chroma_train)

Wall time: 1min 19s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Pickle the trained model

In [178]:
with open('data/interim/{}-1-sec/{}.pkl'.format('chroma_stft', 'logreg-model'), 'wb') as f:
    pickle.dump(chroma_model, f)

##### scoring

Uses the mean class accuracy. See [This link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) on the `.score` method for multiclass classification.

In [180]:
chroma_model.score(X_chroma_test, y_chroma_test)

0.16673701983959477

### mfcc

##### train-test split

In [22]:
# load all features from processed files
train_mfcc = pd.concat([pd.read_pickle('data/interim/{}-1-sec/{}.pkl'.format('mfcc', name)) for name in tags])

df_train_mfcc = (
    train_mfcc
    .merge(file_labels, on=['name'], how='inner')
)

X_mfcc_train, X_mfcc_test, y_mfcc_train, y_mfcc_test = model_selection.train_test_split(
    df_train_mfcc.drop(['name', 'label', 'label_index'], axis=1).values,
    df_train_mfcc['label_index'].values,
    test_size=0.25, 
    random_state=707,
    stratify=df_train_mfcc['label_index'].values
)

Use the multiclass version of logistic regression. I changed to tolerance to 0.001 from 0.0001 because the latter was taking forever (> 30 minutes) to train. 

In [11]:
mfcc_model = linear_model.LogisticRegression(tol=0.001, random_state=123) 

In [12]:
%%time
mfcc_model.fit(X_mfcc_train, y_mfcc_train)

Wall time: 30min 11s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False)

Pickle the trained model

In [13]:
with open('data/interim/{}-1-sec/{}.pkl'.format('mfcc', 'logreg-model'), 'wb') as f:
    pickle.dump(mfcc_model, f)

##### scoring

See [This link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) on the `.score` method for multiclass classification.

In [14]:
mfcc_model.score(X_mfcc_test, y_mfcc_test)

0.28070915998311524

### Stack the two models up

Turn the output from model 1 and 2 into probability predictions.

I used log(probability) instead of raw class probability for no particular reason. 

##### read the trained models

In [17]:
with open('data/interim/{}-1-sec/{}.pkl'.format('chroma_stft', 'logreg-model'), 'rb') as f:
    chroma_model = pickle.load(f)

with open('data/interim/{}-1-sec/{}.pkl'.format('mfcc', 'logreg-model'), 'rb') as f:
    mfcc_model = pickle.load(f)

Note that y_chroma_train and y_mfcc_train are the same - actually I only need to split it once in the preprocessing step. 

In [23]:
np.array_equal(y_chroma_train, y_mfcc_train)

True

#### Use predictions from both models as new features

Because there are 41 classes, each model gives me 41 features. Taking both will give me 41 + 41 = 82 features to train the stacked model.

#### Model 1

In [34]:
X_feature1_train = chroma_model.predict_log_proba(X_chroma_train)
X_feature1_test = chroma_model.predict_log_proba(X_chroma_test)

#### Model 2

In [35]:
X_feature2_train = mfcc_model.predict_log_proba(X_mfcc_train)
X_feature2_test = mfcc_model.predict_log_proba(X_mfcc_test)

In [36]:
display(X_feature1_train.shape)
display(X_feature2_train.shape)

(7104, 41)

(7104, 41)

#### Combine the outputs from model 1 and 2

Combine the features by axis=1

In [40]:
X_train = np.concatenate([X_feature1_train, X_feature2_train], axis=1)
X_test = np.concatenate([X_feature1_test, X_feature2_test], axis=1)
display(X_train.shape)

(7104, 82)

##### train stacked model

In [41]:
stacked_model = linear_model.LogisticRegression(random_state=123)

In [42]:
%%time
stacked_model.fit(X_train, y_mfcc_train)

Wall time: 18.2 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
stacked_model.score(X_test, y_mfcc_test)

0.30434782608695654

### Model performance with mean accuracy

* only chroma_stft features: 16.6%
* only mfcc features: 28.0%
* Stacked logistic regression model using output from chroma_stft and mfcc: 30.4%